In [ ]:
%pip install ipywidgets

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import PIL.Image
import ipywidgets as widgets

In [ ]:
def load_rgb_image(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {path}")
    return img

In [ ]:
img_lena = load_rgb_image('images/lena.png')
img_chips = load_rgb_image('images/chips.png')
img_hsv_disk = load_rgb_image('images/hsv_disk.png')
img_monkey = load_rgb_image('images/monkey.jpeg')
img_strawberries = load_rgb_image('images/strawberries.tif')
img_rgbcube = load_rgb_image('images/rgbcube_kBKG.png')


In [ ]:
images = {
    'lena.png': img_lena,
    'chips.png': img_chips,
    'rgbcube_kBKG.png': img_rgbcube,
    'hsv_disk.png': img_hsv_disk,
    'monkey.jpeg': img_monkey,
    'strawberries.tif': img_strawberries,
}



1. Display Color Histograms for RGB Images


In [ ]:

def plot_rgb_histograms(image, title):
    channels = cv2.split(image)
    colors = ('b', 'g', 'r')

    plt.figure(figsize=(8, 4))
    plt.title(f"RGB Histograms - {title}")
    plt.xlabel("Pixel Value")
    plt.ylabel("Pixel Count")

    for channel, color in zip(channels, colors):
        hist = cv2.calcHist([channel], [0], None, [256], [0, 256])
        plt.plot(hist, color=color)
        plt.xlim([0, 256])

    plt.grid(True)
    plt.tight_layout()
    plt.show()

for file_name, bgr_image in images.items():
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(rgb_image)
    plt.title(f"Original Image - {file_name}")
    plt.axis('off')
    plt.show()
    plot_rgb_histograms(bgr_image, file_name)


2. Visualize Individual Color Channels

In [ ]:


def display_rgb_channels(bgr_image, filename):
    blue, green, red = cv2.split(bgr_image)

    blank = np.zeros_like(blue)
    red_image = cv2.merge([blank, blank, red])
    green_image = cv2.merge([blank, green, blank])
    blue_image = cv2.merge([blue, blank, blank])

    reconstructed = cv2.merge([blue, green, red])

    fig, axes = plt.subplots(2, 4, figsize=(16, 6))

    axes[0, 0].imshow(cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title("Original Image")

    axes[0, 1].imshow(red, cmap='gray')
    axes[0, 1].set_title("Red Channel (grayscale)")

    axes[0, 2].imshow(green, cmap='gray')
    axes[0, 2].set_title("Green Channel (grayscale)")

    axes[0, 3].imshow(blue, cmap='gray')
    axes[0, 3].set_title("Blue Channel (grayscale)")

    axes[1, 0].imshow(cv2.cvtColor(red_image, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title("Red Channel (pseudo-color)")

    axes[1, 1].imshow(cv2.cvtColor(green_image, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title("Green Channel (pseudo-color)")

    axes[1, 2].imshow(cv2.cvtColor(blue_image, cv2.COLOR_BGR2RGB))
    axes[1, 2].set_title("Blue Channel (pseudo-color)")

    axes[1, 3].imshow(cv2.cvtColor(reconstructed, cv2.COLOR_BGR2RGB))
    axes[1, 3].set_title("Reconstructed Image")

    for ax in axes.ravel():
        ax.axis('off')

    plt.suptitle(f'RGB Channel Visualization - {filename}')
    plt.tight_layout()
    plt.show()

for file_name, rgb_img in images.items():
    display_rgb_channels(rgb_img, file_name)


3. Convert Between Color Spaces (RGB ↔ HSV, LAB, YCrCb, CMYK)



In [ ]:


def show_channels(channels, channel_names, base_title):
    plt.figure(figsize=(15, 3))
    for idx, channel in enumerate(channels):
        plt.subplot(1, len(channels), idx + 1)
        plt.imshow(channel, cmap='gray')
        plt.title(f"{base_title} - {channel_names[idx]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

def convert_rgb_to_cmyk(rgb_image):
    pil_img = Image.fromarray(rgb_image)
    cmyk_img = pil_img.convert('CMYK')
    c, m, y, k = cmyk_img.split()
    return np.array(c), np.array(m), np.array(y), np.array(k)

def convert_color_spaces_with_cmyk(bgr_image, file_name):
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    hsv_img = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    show_channels(cv2.split(hsv_img), ['H', 'S', 'V'], f'{file_name} - HSV')

    lab_img = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2LAB)
    show_channels(cv2.split(lab_img), ['L', 'A', 'B'], f'{file_name} - LAB')

    ycrcb_img = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2YCrCb)
    show_channels(cv2.split(ycrcb_img), ['Y', 'Cr', 'Cb'], f'{file_name} - YCrCb')

    c, m, y, k = convert_rgb_to_cmyk(rgb_image)
    show_channels([c, m, y, k], ['C', 'M', 'Y', 'K'], f'{file_name} - CMYK')

for file_name, rgb_img in images.items():
    convert_color_spaces_with_cmyk(rgb_img, file_name)


4. Compare Effects of Blurring in RGB vs HSV



In [ ]:


def apply_blurring(bgr_image, file_name):
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    rgb_blurred = cv2.GaussianBlur(rgb_image, (15, 15), 0)

    hsv_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    hsv_blurred = cv2.GaussianBlur(hsv_image, (15, 15), 0)
    hsv_blurred_bgr = cv2.cvtColor(hsv_blurred, cv2.COLOR_HSV2BGR)

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(rgb_image)
    plt.title(f'Original Image - {file_name}')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(rgb_blurred)
    plt.title(f'Gaussian Blur (RGB) - {file_name}')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(hsv_blurred_bgr, cv2.COLOR_BGR2RGB))
    plt.title(f'Gaussian Blur (HSV) - {file_name}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

for file_name, rgb_img in images.items():
    apply_blurring(rgb_img, file_name)


No espaço RGB, o desfoque gaussiano afeta os três canais de cor (vermelho, verde e azul) de maneira conjunta, o que pode resultar em uma suavização das cores que altera suas relações naturais, fazendo com que a imagem perca a intensidade e a fidelidade das cores originais. Já no espaço HSV, a operação de desfoque é realizada separadamente na componente de luminosidade (Valor - V), enquanto as componentes de matiz (H) e saturação (S) geralmente são preservadas. Isso significa que, ao aplicar o desfoque no espaço HSV, a cor (matiz e saturação) é menos afetada, mantendo a aparência mais fiel e evitando distorções nas tonalidades, o que torna o HSV mais eficiente para preservar as cores em algumas operações.


5. Apply Edge Detection Filters (Sobel, Laplacian) on Color Images



In [ ]:


def apply_edge_filters(bgr_image, file_name):
    gray = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)

    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel_mag = cv2.magnitude(sobel_x, sobel_y)

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)

    sobel_mag = cv2.convertScaleAbs(sobel_mag)
    laplacian = cv2.convertScaleAbs(laplacian)

    rgb_channels = cv2.split(bgr_image)
    sobel_rgb = [cv2.Sobel(ch, cv2.CV_64F, 1, 0, ksize=3) for ch in rgb_channels]
    sobel_rgb_abs = [cv2.convertScaleAbs(s) for s in sobel_rgb]

    laplacian_rgb = [cv2.Laplacian(ch, cv2.CV_64F) for ch in rgb_channels]
    laplacian_rgb_abs = [cv2.convertScaleAbs(l) for l in laplacian_rgb]

    plt.figure(figsize=(15, 10))

    plt.subplot(2, 3, 1)
    plt.imshow(sobel_mag, cmap='gray')
    plt.title(f'Sobel - {file_name} (Grayscale)')
    plt.axis('off')

    plt.subplot(2, 3, 2)
    plt.imshow(laplacian, cmap='gray')
    plt.title(f'Laplacian - {file_name} (Grayscale)')
    plt.axis('off')

    for idx, sobel_channel in enumerate(sobel_rgb_abs):
        plt.subplot(2, 3, 3 + idx)
        plt.imshow(sobel_channel, cmap='gray')
        plt.title(f'Sobel - Channel {["R", "G", "B"][idx]}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

    combined_edges = cv2.bitwise_or(sobel_rgb_abs[0], sobel_rgb_abs[1])
    combined_edges = cv2.bitwise_or(combined_edges, sobel_rgb_abs[2])

    plt.figure(figsize=(6, 6))
    plt.imshow(combined_edges, cmap='gray')
    plt.title(f'Combined Edges (RGB) - {file_name}')
    plt.axis('off')
    plt.show()

for file_name, rgb_image in images.items():
    apply_edge_filters(rgb_image, file_name)


6. High-pass and Low-pass Filtering in the Frequency Domain

In [ ]:


def apply_frequency_filters(bgr_image, file_name):
    gray = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)

    f_transform = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)
    shifted_dft = np.fft.fftshift(f_transform)

    mag_spectrum = np.log(cv2.magnitude(shifted_dft[:, :, 0], shifted_dft[:, :, 1]) + 1)

    height, width = gray.shape
    center_y, center_x = height // 2, width // 2
    radius = 30

    low_pass_mask = np.zeros((height, width), dtype=np.uint8)
    high_pass_mask = np.ones((height, width), dtype=np.uint8)

    cv2.circle(low_pass_mask, (center_x, center_y), radius, 1, -1)
    cv2.circle(high_pass_mask, (center_x, center_y), radius, 0, -1)

    low_filtered = shifted_dft * low_pass_mask[:, :, np.newaxis]
    high_filtered = shifted_dft * high_pass_mask[:, :, np.newaxis]

    low_image = cv2.idft(np.fft.ifftshift(low_filtered))
    high_image = cv2.idft(np.fft.ifftshift(high_filtered))

    low_result = cv2.magnitude(low_image[:, :, 0], low_image[:, :, 1])
    high_result = cv2.magnitude(high_image[:, :, 0], high_image[:, :, 1])

    plt.figure(figsize=(12, 6))

    plt.subplot(2, 3, 1)
    plt.imshow(mag_spectrum, cmap='gray')
    plt.title(f'Magnitude Spectrum - {file_name}')
    plt.axis('off')

    plt.subplot(2, 3, 2)
    plt.imshow(low_pass_mask, cmap='gray')
    plt.title(f'Low-Frequency Mask - {file_name}')
    plt.axis('off')

    plt.subplot(2, 3, 3)
    plt.imshow(low_result, cmap='gray')
    plt.title(f'Low-Frequency Image - {file_name}')
    plt.axis('off')

    plt.subplot(2, 3, 4)
    plt.imshow(high_pass_mask, cmap='gray')
    plt.title(f'High-Frequency Mask - {file_name}')
    plt.axis('off')

    plt.subplot(2, 3, 5)
    plt.imshow(high_result, cmap='gray')
    plt.title(f'High-Frequency Image - {file_name}')
    plt.axis('off')

    plt.subplot(2, 3, 6)
    plt.imshow(gray, cmap='gray')
    plt.title(f'Original Image - {file_name}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

for file_name, rgb_img in images.items():
    apply_frequency_filters(rgb_img, file_name)


7. Visualize and Manipulate Bit Planes of Color Images

In [ ]:
def extract_bit_planes(bgr_image, file_name):
    rgb_channels = cv2.split(bgr_image)
    bit_planes = {}

    for idx, channel in enumerate(rgb_channels):
        msb = channel >> 7
        lsb = channel & 1
        upper_4bits = (channel >> 4) & 15

        label = ["R", "G", "B"][idx]
        bit_planes[f'{label} Channel - MSB'] = msb
        bit_planes[f'{label} Channel - LSB'] = lsb
        bit_planes[f'{label} Channel - Top 4 Bits'] = upper_4bits

    plt.figure(figsize=(12, 8))

    for i, (label, plane) in enumerate(bit_planes.items()):
        plt.subplot(3, 4, i + 1)
        plt.imshow(plane, cmap='gray')
        plt.title(label)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

    reconstructed = cv2.merge([
        bit_planes[f'{c} Channel - Top 4 Bits'] for c in ["R", "G", "B"]
    ])

    plt.figure(figsize=(6, 6))
    plt.imshow(reconstructed)
    plt.title(f'Reconstructed Image - {file_name}')
    plt.axis('off')
    plt.show()

for file_name, rgb_image in images.items():
    extract_bit_planes(rgb_image, file_name)


8. Color-based Object Segmentation using HSV Thresholding

In [ ]:
def color_based_object_segmentation(bgr_image, file_name):
    hsv_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    rgb_display = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    h_range = [0, 179]
    s_range = [100, 255]
    v_range = [100, 255]

    h_min_slider = widgets.IntSlider(value=h_range[0], min=0, max=179, description='H Min:')
    h_max_slider = widgets.IntSlider(value=h_range[1], min=0, max=179, description='H Max:')
    s_min_slider = widgets.IntSlider(value=s_range[0], min=0, max=255, description='S Min:')
    s_max_slider = widgets.IntSlider(value=s_range[1], min=0, max=255, description='S Max:')
    v_min_slider = widgets.IntSlider(value=v_range[0], min=0, max=255, description='V Min:')
    v_max_slider = widgets.IntSlider(value=v_range[1], min=0, max=255, description='V Max:')

    output = widgets.Output()

    def update_mask(h_min, h_max, s_min, s_max, v_min, v_max):
        with output:
            output.clear_output(wait=True)
            lower_bound = np.array([h_min, s_min, v_min])
            upper_bound = np.array([h_max, s_max, v_max])
            mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
            segmented = cv2.bitwise_and(bgr_image, bgr_image, mask=mask)
            segmented_rgb = cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB)

            plt.figure(figsize=(15, 5))
            plt.subplot(1, 3, 1)
            plt.imshow(rgb_display)
            plt.title('Original Image')
            plt.axis('off')
            plt.subplot(1, 3, 2)
            plt.imshow(mask, cmap='gray')
            plt.title('HSV Mask')
            plt.axis('off')
            plt.subplot(1, 3, 3)
            plt.imshow(segmented_rgb)
            plt.title('Segmented Objects')
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            plt.clf()

    sliders = widgets.interactive(update_mask,
        h_min=h_min_slider, h_max=h_max_slider,
        s_min=s_min_slider, s_max=s_max_slider,
        v_min=v_min_slider, v_max=v_max_slider)

    display(h_min_slider, h_max_slider, s_min_slider, s_max_slider, v_min_slider, v_max_slider, output)

    update_mask(*h_range, *s_range, *v_range)

for file_name, rgb_image in images.items():
    color_based_object_segmentation(rgb_image, file_name)


9. Convert and Visualize Images in the NTSC (YIQ) Color Space

In [ ]:
def rgb_to_ntsc(rgb_image):
    normalized = rgb_image / 255.0
    rgb2yiq_matrix = np.array([
        [0.299,  0.587,  0.114],
        [0.596, -0.274, -0.322],
        [0.211, -0.523,  0.312]
    ])
    yiq_image = cv2.transform(normalized, rgb2yiq_matrix.T)
    yiq_image = np.clip(yiq_image, 0, 1)
    return cv2.split(yiq_image)

def show_ntsc_channels(rgb_image, filename):
    Y, I, Q = rgb_to_ntsc(rgb_image)

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    channels = [Y, I, Q]
    titles = ['Y (Luminance)', 'I (In-phase)', 'Q (Quadrature)']

    for ax, channel, title in zip(axes, channels, titles):
        ax.imshow(channel, cmap='gray')
        ax.set_title(f'{filename} - {title}')
        ax.axis('off')

    plt.tight_layout()
    plt.show()

for file_name, rgb_image in images.items():
    show_ntsc_channels(rgb_image, file_name)


10. Color Image Enhancement with Histogram Equalization

In [ ]:
def histogram_equalization(bgr_image, filename):
    ycrcb = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2YCrCb)
    ycrcb_channels = cv2.split(ycrcb)
    ycrcb_channels = list(ycrcb_channels)
    ycrcb_channels[0] = cv2.equalizeHist(ycrcb_channels[0])
    ycrcb_eq = cv2.merge(ycrcb_channels)
    ycrcb_eq_bgr = cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2BGR)

    lab = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2LAB)
    lab_channels = list(cv2.split(lab))
    lab_channels[0] = cv2.equalizeHist(lab_channels[0])
    lab_eq = cv2.merge(lab_channels)
    lab_eq_bgr = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

    images_to_show = [
        (cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB), 'Original'),
        (cv2.cvtColor(ycrcb_eq_bgr, cv2.COLOR_BGR2RGB), 'Equalized (YCrCb)'),
        (cv2.cvtColor(lab_eq_bgr, cv2.COLOR_BGR2RGB), 'Equalized (LAB)')
    ]

    plt.figure(figsize=(12, 6))
    for i, (img, title) in enumerate(images_to_show, 1):
        plt.subplot(1, 3, i)
        plt.imshow(img)
        plt.title(f'{filename} - {title}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

for file_name, rgb_image in images.items():
    histogram_equalization(rgb_image, file_name)


Aplicar a equalização de histograma diretamente no espaço RGB pode resultar em uma imagem com cores distorcidas e não naturais, pois os canais de cor (vermelho, verde e azul) estão muito interligados. Em vez disso, ao trabalhar com espaços de cor como YCrCb ou LAB, onde a luminosidade e a crominância são separadas, podemos melhorar o contraste sem afetar as cores. Por exemplo, ao equalizar o canal de luminosidade (Y no YCrCb ou L no LAB), conseguimos uma imagem com mais detalhes e contraste, mas sem alterar as cores originais, o que preserva a aparência natural da cena.